In [1]:
import pyarrow as pa
data = b'abcdefghijklmnopqrstuvwxyz'
buf = pa.py_buffer(data)
print(buf.size)

26


In [3]:
memoryview(buf)
buf.to_pybytes()

b'abcdefghijklmnopqrstuvwxyz'

In [4]:
buf = memoryview(b"some data")
stream = pa.input_stream(buf)
stream.read(4)

b'some'

In [5]:
with open('./out/example2.dat', 'wb') as f:
    f.write(b'some example data')

In [7]:
file_obj = pa.OSFile('./out/example2.dat')
file_obj.read(4)

b'some'

In [8]:
# Using pyarrow’s OSFile class, you can write:
with pa.OSFile('./out/example3.dat', 'wb') as f:
    f.write(b'some example data')
mmap = pa.memory_map('./out/example3.dat')
mmap.read(4)

b'some'

In [9]:
mmap.seek(0)
buf = mmap.read_buffer(4)
print(buf)

In [10]:
buf.to_pybytes()

b'some'

## In-Memory Reading and Writing
To assist with serialization and deserialization of in-memory data, we have file interfaces that can read and write to Arrow Buffers.

These have similar semantics to Python’s built-in io.BytesIO.

In [11]:
writer = pa.BufferOutputStream()
writer.write(b'hello, friends')
# 14

buf = writer.getvalue()
print(buf)
# <pyarrow.lib.Buffer at 0x2b9df4d9d180>
print(buf.size)
# 14

reader = pa.BufferReader(buf)
reader.seek(7)
reader.read(7)
# b'friends'

14


b'friends'

## Data Types and In-Memory Data Model
Apache Arrow defines columnar array data structures by composing type metadata with memory buffers, like the ones explained in the documentation on Memory and IO. These data structures are exposed in Python through a series of interrelated classes:

* Type Metadata: Instances of pyarrow.DataType, which describe a logical array type
* Schemas: Instances of pyarrow.Schema, which describe a named collection of types. These can be thought of as the column types in a table-like object.
* Arrays: Instances of pyarrow.Array, which are atomic, contiguous columnar data structures composed from Arrow Buffer objects
* Record Batches: Instances of pyarrow.RecordBatch, which are a collection of Array objects with a particular Schema
* Tables: Instances of pyarrow.Table, a logical table data structure in which each column consists of one or more pyarrow.Array objects of the same type.

In [15]:
t1 = pa.int32()
print(t1)
f0 = pa.field('int32_field', t1)
print(f0.name)
print(f0.type)
t6 = pa.list_(t1)
print(t6)

int32
int32_field
int32
list<item: int32>


In [17]:
t1 = pa.int32()
t2 = pa.string()
t3 = pa.binary()
t4 = pa.binary(10)
t5 = pa.timestamp('ms')

fields = [
    pa.field('s0', t1),
    pa.field('s1', t2),
    pa.field('s2', t4),
    pa.field('s3', t6),
]

t7 = pa.struct(fields)
print(t7)

struct<s0: int32, s1: string, s2: fixed_size_binary[10], s3: list<item: int32>>


In [18]:
my_schema = pa.schema([('field0', t1),
                       ('field1', t2),
                       ('field2', t4),
                       ('field3', t6)])

my_schema

field0: int32
field1: string
field2: fixed_size_binary[10]
field3: list<item: int32>
  child 0, item: int32

In [22]:
data = [
    pa.array([1, 2, 3, 4]),
    pa.array(['foo', 'bar', 'baz', None]),
    pa.array([True, None, False, True])
]
batch = pa.RecordBatch.from_arrays(data, ['f0', 'f1', 'f2'])
print(batch.num_columns)
print(batch.num_rows)
print(batch.schema)
print(batch[1])
batch2 = batch.slice(1, 3)
print(batch2[1])

3
4
f0: int64
f1: string
f2: bool
[
  "foo",
  "bar",
  "baz",
  null
]
[
  "bar",
  "baz",
  null
]


## 表
PyArrow Table类型不是Apache Arrow规范的一部分，而是一个帮助将多个记录批次和数组片段作为单个逻辑数据集进行争论的工具。作为一个相关的例子，我们可能在套接字流中接收多个小记录批次，然后需要将它们连接成连续的内存以便在NumPy或pandas中使用。Table对象使这种方法更有效，而无需额外的内存复制。

考虑到我们上面创建的记录批次，我们可以使用以下方法创建一个包含批次的一个或多个副本的表Table.from_batches：

In [24]:
batches = [batch] * 5
table = pa.Table.from_batches(batches)
print(table)
print(table.num_rows)
print(table.num_columns)

pyarrow.Table
f0: int64
f1: string
f2: bool
20
3


In [26]:
# 表的列是实例Column，它是一个或多个相同类型的数组的容器。
c = table[0]
c

<Column name='f0' type=DataType(int64)>
[
  [
    1,
    2,
    3,
    4
  ],
  [
    1,
    2,
    3,
    4
  ],
  [
    1,
    2,
    3,
    4
  ],
  [
    1,
    2,
    3,
    4
  ],
  [
    1,
    2,
    3,
    4
  ]
]

In [27]:
print(c.data)
print(c.data.num_chunks)

[
  [
    1,
    2,
    3,
    4
  ],
  [
    1,
    2,
    3,
    4
  ],
  [
    1,
    2,
    3,
    4
  ],
  [
    1,
    2,
    3,
    4
  ],
  [
    1,
    2,
    3,
    4
  ]
]
5


In [28]:
print(c.data.chunk(0))

[
  1,
  2,
  3,
  4
]


In [29]:
# 可以将这些对象转换为连续的NumPy数组，以便在pandas中使用：
c.to_pandas()

0     1
1     2
2     3
3     4
4     1
5     2
6     3
7     4
8     1
9     2
10    3
11    4
12    1
13    2
14    3
15    4
16    1
17    2
18    3
19    4
Name: f0, dtype: int64

In [30]:
# pyarrow.concat_tables如果模式相同，也可以将多个表连接在一起以形成单个表 ：
tables = [table] * 2
table_all = pa.concat_tables(tables)
table_all.num_rows

40

In [31]:
c = table_all[0]
c.data.num_chunks

10

In [67]:
df_new = table_all.to_pandas()
df_new[:5]

,f0,f1,f2
0,1,foo,True
1,2,bar,None
2,3,baz,False
3,4,None,True
4,1,foo,True


这类似于Table.from_batches，但使用表作为输入而不是记录批次。记录批次可以制作成表格，但不是相反，所以如果您的数据已经是表格形式，那么请使用 pyarrow.concat_tables。

## Arrow定义了两种用于序列化记录批次的二进制格式：

- 流格式：用于发送任意长度的记录批次序列。必须从头到尾处理格式，并且不支持随机访问
- 文件或随机访问格式：用于序列化固定数量的记录批次。支持随机访问，因此在与内存映射一起使用时非常有用

In [32]:
import pyarrow as pa

data = [
    pa.array([1, 2, 3, 4]),
    pa.array(['foo', 'bar', 'baz', None]),
    pa.array([True, None, False, True])
]

batch = pa.RecordBatch.from_arrays(data, ['f0', 'f1', 'f2'])
batch.num_rows

4

In [36]:
# 现在，我们可以开始编写包含一些批次的流。为此我们使用RecordBatchStreamWriter，
# 它可以写入可写 NativeFile对象或可写Python对象：
# 这里我们使用了内存中的Arrow缓冲流，但这可能是一个套接字或其他一些IO接收器。
sink = pa.BufferOutputStream()
writer = pa.RecordBatchStreamWriter(sink, batch.schema)

for i in range(5):
    writer.write_batch(batch)

writer.close()
buf = sink.getvalue()
print(buf.size)
print(type(buf))

1972
<class 'pyarrow.lib.Buffer'>


In [37]:
reader = pa.ipc.open_stream(buf)
print(reader.schema)

batches = [b for b in reader]
len(batches)

f0: int64
f1: string
f2: bool


5

In [38]:
# 检查返回的批次是否与原始输入相同：
batches[0].equals(batch)

True

In [39]:
# 在RecordBatchFileWriter具有相同的API RecordBatchStreamWriter：
sink = pa.BufferOutputStream()
writer = pa.RecordBatchFileWriter(sink, batch.schema)

for i in range(10):
    writer.write_batch(batch)

writer.close()

buf = sink.getvalue()
buf.size

4210

RecordBatchFileReader和 之间的区别在于RecordBatchStreamReader输入源必须具有seek随机访问的 方法。流读取器仅需要读取操作。我们也可以使用该pyarrow.ipc.open_file方法打开一个文件：

In [40]:
reader = pa.ipc.open_file(buf)
reader.num_record_batches
# 10
b = reader.get_batch(3)

In [42]:
# 流和文件阅读器类有一个特殊的read_pandas方法来简化读取多个记录批次并将它们转换为单个DataFrame输出
df = pa.ipc.open_file(buf).read_pandas()
df[:5]

,f0,f1,f2
0,1,foo,True
1,2,bar,None
2,3,baz,False
3,4,None,True
4,1,foo,True


## 任意对象序列化
在pyarrow我们能够序列化和反序列化多种Python对象。虽然不是pickle模块的完全替代品，但这些功能可以明显更快，特别是在处理NumPy阵列的集合时。

In [43]:
import numpy as np

data = {
    i: np.random.randn(500, 500)
    for i in range(100)
}

In [44]:
buf = pa.serialize(data).to_buffer()
print(type(buf))
print(buf.size)

<class 'pyarrow.lib.Buffer'>
200028864


pyarrow.serialize创建一个中间对象，可以将其转换为缓冲区（to_buffer方法）或直接写入输出流。
pyarrow.deserialize 将类缓冲区对象转换回原始Python对象：
处理NumPy数组时，pyarrow.deserialize可能会明显快于pickle因为生成的数组是对输入缓冲区的零拷贝引用。阵列越大，性能节省越多。

In [45]:
restored_data = pa.deserialize(buf)
restored_data[0]

array([[ 0.12742827, -0.95459129,  0.05318619, ..., -0.74436129,
         0.96213285, -0.42219318],
       [-2.0200782 , -1.33166671, -1.49418935, ...,  0.29310029,
        -0.95510217, -0.31432893],
       [-0.25230722, -0.24753379, -2.61528964, ...,  0.70687883,
         0.68887424, -0.27028108],
       ...,
       [ 1.4241211 , -0.44546557, -0.11266728, ..., -0.07963118,
        -0.41794279,  0.60465669],
       [ 0.55870002,  0.90073452, -0.34766403, ...,  0.15999091,
         1.78429767, -1.32387707],
       [-1.84781187,  0.80243619,  0.31178591, ..., -0.12645207,
         0.38196331, -0.13160539]])

In [46]:
%timeit restored_data = pa.deserialize(buf)

509 µs ± 14.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [47]:
import pickle
pickled = pickle.dumps(data)
%timeit unpickled_data = pickle.loads(pickled)

133 ms ± 8.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
import pyarrow as pa

# 考虑一个有两个成员的类，其中一个是NumPy数组：
class MyData:
    def __init__(self, name, data):
        self.name = name
        self.data = data

# 我们编写函数来将它转换为具有更简单类型的字典：
def _serialize_MyData(val):
    return {'name': val.name, 'data': val.data}

def _deserialize_MyData(data):
    return MyData(data['name'], data['data'])


In [58]:
# 那么，我们必须在注册这些功能SerializationContext，这样 MyData可以确认：
context=pa.SerializationContext()
context.register_type(MyData, 'MyData',
                      custom_serializer=_serialize_MyData,
                      custom_deserializer=_deserialize_MyData)

val=MyData("hello", "world")
# 最后，我们使用此上下文作为附加参数pyarrow.serialize：
buf = pa.serialize(val, context=context).to_buffer()
restored_val = pa.deserialize(buf, context=context)
# 该SerializationContext还具有方便的方法serialize和 deserialize，所以这些都是等效的声明：
buf = context.serialize(val).to_buffer()
restored_val = context.deserialize(buf)
print(restored_val, restored_val.data)

<__main__.MyData object at 0x10f9f47b8> world


## 基于组件的序列化
对于序列化包含一定数量的NumPy数组，箭头缓冲区或其他数据类型的Python对象，可能需要传输其序列化表示而不必使用该to_buffer方法生成中间副本 。为了激发这一点，假设我们有一个NumPy数组列表：..

该调用pa.serialize(data)不会复制每个NumPy数组中的内存。然后，可以将此序列化表示分解为包含一系列pyarrow.Buffer对象的字典，该对象包含每个数组的元数据以及对数组内部内存的引用。为此，请使用以下to_components方法：

In [59]:
import numpy as np
data = [np.random.randn(10, 10) for i in range(5)]

serialized = pa.serialize(data)
components = serialized.to_components()

In [60]:
memoryview(components['data'][0])

In [61]:
# memoryview可以转换回箭Buffer与 pyarrow.py_buffer：
mv = memoryview(components['data'][0])
buf = pa.py_buffer(mv)


In [62]:
# 使用以下方法从基于组件的表示重建对象 deserialize_components：
restored_data = pa.deserialize_components(components)
restored_data[0]

array([[-1.50122557, -1.19676195,  0.1266847 , -1.85551048, -0.86232847,
        -1.36439755,  0.32611734, -1.38306278, -0.99390007,  1.39160018],
       [ 1.61327357, -0.25970142, -0.20182893,  0.08072281,  0.60007608,
         0.28938348,  1.2834959 , -0.63898964, -1.47821378,  1.56514748],
       [ 0.92313537, -1.14870166,  2.60787796,  0.95204378, -1.54519512,
        -1.02754663,  1.79460792,  0.53590907,  0.39266188, -0.57042973],
       [-1.93516557,  1.049958  ,  0.61075164,  1.61545997,  0.22881206,
         1.26159629,  1.87400733, -1.20716317, -1.80240749, -0.22434756],
       [ 0.16278112, -0.7571062 ,  1.10146151,  0.13153447, -1.04254708,
         1.00061581, -1.07659186,  0.14857146, -0.24341449, -1.09934827],
       [ 2.5084868 ,  1.81081776, -0.01036504,  2.24718117,  0.51473965,
        -0.46864375,  0.09480174,  0.4545933 ,  0.82106186,  0.50150186],
       [ 0.48789061,  0.54642272,  0.72467134,  1.47021037, -1.93137476,
        -0.22946273,  3.05576351, -0.3826343 

## 序列化pandas对象
默认序列化方面进行了优化像熊猫的对象处理DataFrame和Series。结合上面的基于组件的序列化，这可以实现不包含任何Python对象的pandas DataFrame对象的零拷贝传输：

In [64]:
import pandas as pd

df = pd.DataFrame({'a': [1, 2, 3, 4, 5]})
context = pa.default_serialization_context()

serialized_df = context.serialize(df)
df_components = serialized_df.to_components()
original_df = context.deserialize_components(df_components)
original_df

,a
0,1
1,2
2,3
3,4
4,5
